In [2]:
import pickle

In [3]:
# Load episode data from previous scraping exercise
with open('simpsons_scripts.pickle', 'rb') as eps_file:
    episodes = pickle.load(eps_file)

In [4]:
# Go through the seasons and add scripts to a list to facilitate vectorization
ep_scripts = []
ep_script_lookup = {}
for season in episodes:
    for episode in episodes[season]:
        ep_scripts.append(episodes[season][episode]['script'])
        ep_script_lookup[len(ep_scripts)-1] = {}
        ep_script_lookup[len(ep_scripts)-1]['season'] = season
        ep_script_lookup[len(ep_scripts)-1]['title'] = episode

In [5]:
# Use Scikit-Learn TF-IDF feature
from sklearn.feature_extraction.text import TfidfVectorizer

# TODO: experiment with parameters
tfidf_vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, ngram_range=(1,3), min_df=2, max_df=0.5)
X = tfidf_vectorizer.fit_transform(ep_scripts)
X_array = X.toarray()

In [6]:
# Take a peek at how the results are

episode = 37
print(ep_script_lookup[episode])
ep_max = X_array[episode].argmax()
tfidf_vectorizer.get_feature_names()[ep_max]

{'season': 3, 'title': 3}


'left handed'